In [2]:
# Deze branch gaat niet toegankelijk gemaakt worden voor Oorschot! Better remove it,
# maybe plagiaat-check sees this one!!!

# ALLES NEDERLANDS
# Hoe meer in verslag, hoe minder % plagiaat

# Verander namen van alle variabelen.
# Versimpel code naar ons niveau.
# Limit informational blogs.
# Andere databases gebruiken
# Deliver our projects NOT in notebooks for convincing? But if we use -> Explain!
# Change use of functions! -> more and different function use -> won't be checked by plagiaat-check
# Could fuck up the plagiaat algorithm with random comments and characters?
# Get rid of the function annotations
# Follow informatica codeer-regels
# Check informatica beoordeling formulier
# Variable name design with capital letters not underscores
# Bronvermelding
# Consider using only Numpy instead of Pandas
# Explain why we use RNN

# Change order. Delete redundant. Replace some maybe? Explain every module.


import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

2022-03-18 14:55:32.392433: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 14:55:32.392465: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Review later

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

_SAMPLING_RATE = 16000

In [4]:
# Choice for Path, since we use Linux and Windows, downloaded MAESTRO, extract, if not data_dir -> unnecessary!
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [5]:
# Choosing between this or with for-loop (see test)

filenames = []
for i in glob.glob('data/maestro-v2.0.0/*/*[.mid]*'):
    filenames.append(i)

filenames = glob.glob(str(data_dir/'**/*.midi*'))


In [6]:
# Consider adding Linux download of file.

sample_file = filenames[1]
print(sample_file)


data/maestro-v2.0.0/2018/MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--3.midi


In [7]:
pm = pretty_midi.PrettyMIDI(sample_file)


In [8]:
# unnecessary

def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [9]:
# unnecessary

display_audio(pm)


fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable


In [10]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)


Number of instruments: 1
Instrument name: Acoustic Grand Piano


In [11]:
# Simplify enumerate()?

for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start
  print(f'{i}: pitch={note.pitch}, note_name={note_name},'
        f' duration={duration:.4f}')


0: pitch=45, note_name=A2, duration=0.3945
1: pitch=50, note_name=D3, duration=0.4089
2: pitch=57, note_name=A3, duration=0.4453
3: pitch=53, note_name=F3, duration=0.6133
4: pitch=38, note_name=D2, duration=0.2500
5: pitch=62, note_name=D4, duration=0.2240
6: pitch=65, note_name=F4, duration=0.3151
7: pitch=57, note_name=A3, duration=0.1745
8: pitch=45, note_name=A2, duration=0.1523
9: pitch=50, note_name=D3, duration=0.1966


In [12]:
# Simplify: default-dict -> dict or list ? Why use pd.DataFrame? No alternative? -> Numpy?
# Declare and assign ndarray before the return keyword.
# Simplify the return value of this function

def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})


In [13]:
raw_notes = midi_to_notes(sample_file)
#raw_notes.head()


In [14]:
# Is velocity necessary?

def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness (source: tensorfow.org tutorial)
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [20]:
num_files = 5
all_notes = []
for f in filenames[:num_files]:
  notes = midi_to_notes(f)
  all_notes.append(notes)

all_notes = pd.concat(all_notes)

In [21]:
# Unnecessary

n_notes = len(all_notes)
print('Number of notes parsed:', n_notes)


Number of notes parsed: 58460


In [25]:
key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)
print(all_notes['pitch'])


0        47
1        66
2        74
3        71
4        66
         ..
16216    27
16217    39
16218    51
16219    27
16220    39
Name: pitch, Length: 58460, dtype: int64


In [29]:
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

TensorSpec(shape=(3,), dtype=tf.float64, name=None)